Coire Gavin-Hanner
Feb 26, 2019

# Overview
This tutorial will guide you through the addition of a new residue to the gromacs charmm36 force field. The example will use the holo acyl carrier protein from geobacter Metallireducens (GmACPholo) which has a 4'-PHOSPHOPANTETHEINE arm attached to a serine residue. In this example, PNS will refer to the arm attached to the serine residue (PNS = arm + serine).

<img src="Example_PNS.png" alt="2lml" width="600"/>

This tutorial also assumes basic knowledge of gromacs. If you are new to gromacs please follow this [tutorial](http://www.mdtutorials.com/gmx/lysozyme/index.html) that sets up a simulation with Lysozyme in water. 

Throughout this document, when working on a specific file, I included an example of what that file should look like. I have also included an example of the finished produc ([Example_charmm36-mar2019.ff](Example_charmm36-mar2019.ff))

## Charmm36 and Gromacs
Before going over what our steps will be, I should briefly explain how gromacs uses the charmm36 force field. The files in the force field function together to define the structure of residues along with the parameters for each atom in the residues. The parameters include non-bonded (atom types and charges) and bonded parameters (bonds, angles, dihedrals, and improper dihedrals).  

The first important file is atomtypes.atp. This lists every atom type that is in a residue in charmm36. An atom type describes the state of the atom in question. For example, the P atom in a negatively charged phosphate will have the atom type "PG1" while a P atom in a neutral phosphate will have the atom type "PG0". Listed next to each atom type is the mass of that type. 

The next file is merged.rtp. It defines every residue in charmm36. Every residue record contains lists of the atoms, bonds, improper dihedral angles, and torsion correctional map (CMAP) parameters in the residue. The atoms list contains all the non-bonded parameters in the residue: atom names, atom types, and charges. The bonds, improper dihedrals, and cmap parameters are then listed by atom name. 

ffbonded.itp contains all bonded parameter information. It lists Bonds, Angles, Dihedrals, and improper dihedral angles. All parameters are defined by atom type not atom name. To find a parameter listed in the residue definition, translate the atom names to their atom types and search in ffbonded.itp. 

The residue definition is used to create the topology file while the parameter informaiton is used to create an atomic-level input (.tpr) file which contains all paramter information for all the atoms of the protein. 

Our goal is to define the new residue, PNS, within the charmm36 files. We will do this by using the Charmm General Force Field (CGenff) program. CGenff takes in a .mol2 structure file and performs atom typing and parameter assignment by analogy. 

_**Note:**_ when passing the new residue to CGenff, the programm does not know that it will be attached to a protein. Therefore, modifications must be made to the parameters that it gives us. If you are looking to simulate a protein-ligand complex where the ligand is not covalently bonded to the protein, try this [tutorial](http://www.mdtutorials.com/gmx/complex/index.html) by Justin Lemkul, PhD

# Prepare Protein and arm pdb files 

Download pdb file from [the protein data bank](https://www.rcsb.org/structure/2lml)

In [ ]:
import py3Dmol; cut=1; pdbdata=open('Example_2lml.pdb', 'r').read(); view=py3Dmol.view(); view.addModels(pdbdata,'pdb'); view.setStyle({'cartoon':{'color':'orange'}}); view.zoomTo(); view

Open the file in Chimera. Often, as shown above, the pdb file will consist of multiple overlapping models of the protein. We want to save only one model. 
- File > Save PDB
- There will be a list of models under "Save Models: " choose one and save file as GmACP_holo
- Open and inspect this new pdb file in chimera, if everything is correct proceed to the next step
- If using ChimeraX
    - Select all protein chains
    - de-select the chain you want to keep
    - delete all the other atoms
    - save the result as a PDB file

In [ ]:
import py3Dmol; cut=1; pdbdata=open('Example_GmACP_holo.pdb', 'r').read(); view=py3Dmol.view(); view.addModels(pdbdata,'pdb'); view.setStyle({'cartoon':{'color':'orange'}}); view.zoomTo(); view

### Fix unresolved atoms and residues in the pdb file using profix. 
The profix man page can be found [here](https://honiglab.c2b2.columbia.edu/software/Jackal/Jackalmanual.htm#profix). The options -prm 2 tells profix to use the heavy atom model which strips protons from all the residues in the protein. -fix 1 tells profix to recover missing atoms and residues. Missing residues are defined as existing in the SEQRES card, but not the ATOM card. This will produce GmACP_holo_fix.pdb

In [ ]:
! profix -prm 2 -fix 1 GmACP_holo.pdb

[Result](Example_GmACP_holo_fix.pdb)

### Edit the protein pdb file

Copy the SER atoms from GmACP_holo.pdb to GmACP_holo_fix.pdb. We need the Hydrogen atoms in that residue that were deleted by profix.

[Result](Example2_GmACP_holo_fix.pdb)

So far, the pdb file has been treating the arm like a ligand that is attached to a residue (SER). We want the arm to be a part of a residue called PNS which we will define in later steps. 

In [ ]:
! cp GmACP_holo_fix.pdb GmACP_PNS36.pdb

Open GmACP_PNS36.pdb. Look for the LINK card which tells you where the arm is attached to the protein. In this case, Ser 36:
```
...
HELIX    3   3 ASP A   35  TYR A   50  1                                  16
HELIX    4   4 ALA A   55  GLN A   60  1                                   6
HELIX    5   5 ASN A   64  GLU A   81  1                                  18
LINK        OG   SER A  36                P24  PNS A  88     1555   1555  1.62
SITE     1 AC1  8 ALA A  28  TRP A  34  ASP A  35  SER A  36                    
SITE     2 AC1  8 HIS A  39  LEU A  62  LYS A  63  LEU A  68                    
CRYST1    1.000    1.000    1.000  90.00  90.00  90.00 P 1           1   
...
```

At the Bottom of the .pdb file, there will be a list of atoms in the HETATM card (shown below). These are the atoms that make up the PNS arm. 
```
...
ATOM      0  CB  HIS A  87       8.305   6.339  13.692  1.00  0.00    GmACP_holo
ATOM      0  CG  HIS A  87       8.589   5.024  13.023  1.00  0.00    GmACP_holo
ATOM      0  ND1 HIS A  87       7.684   4.384  12.217  1.00  0.00    GmACP_holo
ATOM      0  CD2 HIS A  87       9.690   4.228  13.045  1.00  0.00    GmACP_holo
ATOM      0  CE1 HIS A  87       8.201   3.258  11.783  1.00  0.00    GmACP_holo
ATOM      0  NE2 HIS A  87       9.416   3.138  12.267  1.00  0.00    GmACP_holo
TER
HETATM 1419  P24 PNS A  88      10.344  11.968 -10.325  1.00  0.00           P
HETATM 1420  O25 PNS A  88      10.730  11.152 -11.488  1.00  0.00           O
HETATM 1421  O26 PNS A  88      10.381  13.445 -10.479  1.00  0.00           O
HETATM 1422  O27 PNS A  88       8.932  11.475  -9.737  1.00  0.00           O
HETATM 1423  C28 PNS A  88       7.694  11.634 -10.423  1.00  0.00           C
HETATM 1424  C29 PNS A  88       6.706  10.492 -10.006  1.00  0.00           C
HETATM 1425  C30 PNS A  88       5.303  10.781 -10.597  1.00  0.00           C
HETATM 1426  C31 PNS A  88       6.580  10.470  -8.459  1.00  0.00           C
HETATM 1427  C32 PNS A  88       7.283   9.094 -10.530  1.00  0.00           C
HETATM 1428  O33 PNS A  88       8.527   8.789  -9.871  1.00  0.00           O
HETATM 1429  C34 PNS A  88       6.324   7.909 -10.305  1.00  0.00           C
HETATM 1430  O35 PNS A  88       6.548   7.091  -9.405  1.00  0.00           O
HETATM 1431  N36 PNS A  88       5.278   7.822 -11.147  1.00  0.00           N
HETATM 1432  C37 PNS A  88       4.259   6.775 -11.092  1.00  0.00           C
HETATM 1433  C38 PNS A  88       3.487   6.799  -9.777  1.00  0.00           C
HETATM 1434  C39 PNS A  88       2.723   8.097  -9.478  1.00  0.00           C
HETATM 1435  O40 PNS A  88       2.618   8.997 -10.320  1.00  0.00           O
HETATM 1436  N41 PNS A  88       2.144   8.147  -8.271  1.00  0.00           N
HETATM 1437  C42 PNS A  88       1.371   9.301  -7.787  1.00  0.00           C
HETATM 1438  C43 PNS A  88       0.400   8.889  -6.684  1.00  0.00           C
HETATM 1439  S44 PNS A  88       1.244   8.118  -5.298  1.00  0.00           S
HETATM 1440 H282 PNS A  88       7.278  12.602 -10.150  1.00  0.00           H
HETATM 1441 H281 PNS A  88       7.857  11.606 -11.496  1.00  0.00           H
HETATM 1442 H303 PNS A  88       4.590  10.063 -10.187  1.00  0.00           H
HETATM 1443 H302 PNS A  88       4.984  11.783 -10.332  1.00  0.00           H
HETATM 1444 H301 PNS A  88       5.326  10.685 -11.673  1.00  0.00           H
HETATM 1445 H313 PNS A  88       6.200  11.422  -8.102  1.00  0.00           H
HETATM 1446 H312 PNS A  88       5.893   9.686  -8.158  1.00  0.00           H
HETATM 1447 H311 PNS A  88       7.549  10.281  -8.014  1.00  0.00           H
HETATM 1448  H32 PNS A  88       7.478   9.171 -11.593  1.00  0.00           H
HETATM 1449  H33 PNS A  88       8.336   8.332  -9.041  1.00  0.00           H
HETATM 1450  H36 PNS A  88       5.167   8.505 -11.821  1.00  0.00           H
HETATM 1451 H372 PNS A  88       3.565   6.921 -11.912  1.00  0.00           H
HETATM 1452 H371 PNS A  88       4.740   5.807 -11.199  1.00  0.00           H
HETATM 1453 H382 PNS A  88       2.778   5.984  -9.778  1.00  0.00           H
HETATM 1454 H381 PNS A  88       4.200   6.641  -8.982  1.00  0.00           H
HETATM 1455  H41 PNS A  88       2.240   7.379  -7.675  1.00  0.00           H
HETATM 1456 H422 PNS A  88       2.057  10.045  -7.397  1.00  0.00           H
HETATM 1457 H421 PNS A  88       0.807   9.730  -8.610  1.00  0.00           H
HETATM 1458 H431 PNS A  88      -0.305   8.172  -7.085  1.00  0.00           H
HETATM 1459 H432 PNS A  88      -0.138   9.754  -6.325  1.00  0.00           H
HETATM 1460  H44 PNS A  88       2.116   8.987  -4.819  1.00  0.00           H
CONECT  549 1419  548
CONECT 1419  549 1420 1421 1422
CONECT 1420 1419
CONECT 1421 1419
CONECT 1422 1419 1423
CONECT 1423 1422 1424 1440 1441
...
```


Copy these HETATM lines into the ATOM card where the arm is attached (SER 36) as shown below
```
...
ATOM      0  CG  ASP A  35       9.426  16.784  -7.164  1.00  0.00    GmACP_holo
ATOM      0  OD1 ASP A  35       8.859  17.532  -6.352  1.00  0.00    GmACP_holo
ATOM      0  OD2 ASP A  35      10.610  16.414  -7.001  1.00  0.00    GmACP_holo
ATOM      0  N   SER A  36       9.436  13.498  -7.773  1.00  0.00    GmACP_holo
ATOM      0  CA  SER A  36      10.230  12.421  -7.122  1.00  0.00    GmACP_holo
ATOM      0  C   SER A  36      10.540  12.726  -5.643  1.00  0.00    GmACP_holo
ATOM      0  O   SER A  36      10.269  11.906  -4.765  1.00  0.00    GmACP_holo
ATOM      0  CB  SER A  36      11.563  12.194  -7.914  1.00  0.00    GmACP_holo
ATOM      0  OG  SER A  36      11.412  11.533  -9.190  1.00  0.00    GmACP_holo
ATOM      0  H   SER A  36       9.754  13.800  -8.639  1.00  0.00    GmACP_holo
ATOM      0  HA  SER A  36       9.646  11.508  -7.177  1.00  0.00    GmACP_holo
ATOM      0  HB2 SER A  36      12.036  13.152  -8.091  1.00  0.00    GmACP_holo
ATOM      0  HB3 SER A  36      12.235  11.588  -7.315  1.00  0.00    GmACP_holo
HETATM 1419  P24 PNS A  88      10.344  11.968 -10.325  1.00  0.00           P
HETATM 1420  O25 PNS A  88      10.730  11.152 -11.488  1.00  0.00           O
HETATM 1421  O26 PNS A  88      10.381  13.445 -10.479  1.00  0.00           O
HETATM 1422  O27 PNS A  88       8.932  11.475  -9.737  1.00  0.00           O
HETATM 1423  C28 PNS A  88       7.694  11.634 -10.423  1.00  0.00           C
HETATM 1424  C29 PNS A  88       6.706  10.492 -10.006  1.00  0.00           C
HETATM 1425  C30 PNS A  88       5.303  10.781 -10.597  1.00  0.00           C
HETATM 1426  C31 PNS A  88       6.580  10.470  -8.459  1.00  0.00           C
HETATM 1427  C32 PNS A  88       7.283   9.094 -10.530  1.00  0.00           C
HETATM 1428  O33 PNS A  88       8.527   8.789  -9.871  1.00  0.00           O
HETATM 1429  C34 PNS A  88       6.324   7.909 -10.305  1.00  0.00           C
HETATM 1430  O35 PNS A  88       6.548   7.091  -9.405  1.00  0.00           O
HETATM 1431  N36 PNS A  88       5.278   7.822 -11.147  1.00  0.00           N
HETATM 1432  C37 PNS A  88       4.259   6.775 -11.092  1.00  0.00           C
HETATM 1433  C38 PNS A  88       3.487   6.799  -9.777  1.00  0.00           C
HETATM 1434  C39 PNS A  88       2.723   8.097  -9.478  1.00  0.00           C
HETATM 1435  O40 PNS A  88       2.618   8.997 -10.320  1.00  0.00           O
HETATM 1436  N41 PNS A  88       2.144   8.147  -8.271  1.00  0.00           N
HETATM 1437  C42 PNS A  88       1.371   9.301  -7.787  1.00  0.00           C
HETATM 1438  C43 PNS A  88       0.400   8.889  -6.684  1.00  0.00           C
HETATM 1439  S44 PNS A  88       1.244   8.118  -5.298  1.00  0.00           S
HETATM 1440 H282 PNS A  88       7.278  12.602 -10.150  1.00  0.00           H
HETATM 1441 H281 PNS A  88       7.857  11.606 -11.496  1.00  0.00           H
HETATM 1442 H303 PNS A  88       4.590  10.063 -10.187  1.00  0.00           H
HETATM 1443 H302 PNS A  88       4.984  11.783 -10.332  1.00  0.00           H
HETATM 1444 H301 PNS A  88       5.326  10.685 -11.673  1.00  0.00           H
HETATM 1445 H313 PNS A  88       6.200  11.422  -8.102  1.00  0.00           H
HETATM 1446 H312 PNS A  88       5.893   9.686  -8.158  1.00  0.00           H
HETATM 1447 H311 PNS A  88       7.549  10.281  -8.014  1.00  0.00           H
HETATM 1448  H32 PNS A  88       7.478   9.171 -11.593  1.00  0.00           H
HETATM 1449  H33 PNS A  88       8.336   8.332  -9.041  1.00  0.00           H
HETATM 1450  H36 PNS A  88       5.167   8.505 -11.821  1.00  0.00           H
HETATM 1451 H372 PNS A  88       3.565   6.921 -11.912  1.00  0.00           H
HETATM 1452 H371 PNS A  88       4.740   5.807 -11.199  1.00  0.00           H
HETATM 1453 H382 PNS A  88       2.778   5.984  -9.778  1.00  0.00           H
HETATM 1454 H381 PNS A  88       4.200   6.641  -8.982  1.00  0.00           H
HETATM 1455  H41 PNS A  88       2.240   7.379  -7.675  1.00  0.00           H
HETATM 1456 H422 PNS A  88       2.057  10.045  -7.397  1.00  0.00           H
HETATM 1457 H421 PNS A  88       0.807   9.730  -8.610  1.00  0.00           H
HETATM 1458 H431 PNS A  88      -0.305   8.172  -7.085  1.00  0.00           H
HETATM 1459 H432 PNS A  88      -0.138   9.754  -6.325  1.00  0.00           H
HETATM 1460  H44 PNS A  88       2.116   8.987  -4.819  1.00  0.00           H
ATOM      0  N   LEU A  37      11.090  13.918  -5.391  1.00  0.00    GmACP_holo
ATOM      0  CA  LEU A  37      11.522  14.348  -4.045  1.00  0.00    GmACP_holo
ATOM      0  C   LEU A  37      10.303  14.451  -3.096  1.00  0.00    GmACP_holo
ATOM      0  O   LEU A  37      10.344  13.948  -1.972  1.00  0.00    GmACP_holo
ATOM      0  CB  LEU A  37      12.304  15.700  -4.153  1.00  0.00    GmACP_holo
ATOM      0  CG  LEU A  37      13.410  16.010  -3.081  1.00  0.00    GmACP_holo
...
```

In [ ]:
# These commands will edit the PDB file as outlined above
! sed -i 's/HETATM [0-9][0-9][0-9][0-9]/ATOM      0/' GmACP_PNS36.pdb
! sed -i 's/SER A  36/PNS A  36/' GmACP_PNS36.pdb
! sed -i 's/PNS A  88/PNS A  36/' GmACP_PNS36.pdb
! sed -i 's/           ./    GmACP_holo/' GmACP_PNS36.pdb

delete everything except the ATOM lines. Everythign else is unnecessary 

[Result](Example_GmACP_PNS36.pdb)

In [ ]:
import py3Dmol; cut=1; pdbdata=open('Example_GmACP_PNS36.pdb', 'r').read(); view=py3Dmol.view(); view.addModels(pdbdata,'pdb'); view.setStyle({'cartoon':{'color':'orange'}}); view.zoomTo(); view

### Create the Arm PDB file

- open GmACP_holo.pdb in Chimera (This is the first pdb file that we saved, before we used profix)
- Open the command line (Favorites > Command Line)
- Delete residues 1-33 and 39-87
    - "sel :1-33, 39-87" in the command line
    - Actions > Atoms/Bonds > Delete
- Save as PNS.pdb
- Open the pdb file in a text editor and change all atom names in residues 34, 35, 37, and 38 to "X"
    - leave alone the atoms in residue 36 and PPant arm
    - this will be a tedious process, but will make things easier later on by allowing you to easily identify the atoms in the PNS residue
    
[**Result**](Example_PNS.pdb)

**NOTE: The two residues on either side of the residue of interest act like buffers. CGenFF thinks that what the molecule passed to it is a discrete molecule. It does not give us the exact atom types and charges that we need at the ends of the molecule. We get around this by including the buffer residues. Because the changes that we are making occur many atoms away from the important section (the PPant arm) CGenff gives us atom charges that are closer to what we need.**

# Create Arm Parameters

- Download the current and old forcefield files from the [charmm36 site ](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs)
    - Do this on your local machine and then transfer to the WSx workstation as a .tgz file

In [ ]:
! tar -xzvf "current_ff_file.ff.tgz" 
! tar -xzvf "old_ff_file.ff.tgz"

In [ ]:
! cp /packages/gromacs5.1.2/share/gromacs/top/residuetypes.dat ./

### Upload the arm to CGenff

Open PNS.pdb in [Avogadro](https://sourceforge.net/projects/avogadro/)
- _**Note: This step requires a lot of careful work. One atom out of place here will cause a massive headache for you later as you try to troubleshoot the resulting problems**_
- Build > Add Hydrogens 
    - without this step, the terminal nitrogen is missing a hydrogen and will cause errors later
    - This may also add hydrogens to the Ppant arm. If this happens delete them
        - Ex:When doing this, I found that Avogadro added a hydrogen to O26 of the Ppant arm
    - If you delete atoms using a text editor, update the atom and bond numbers in the top line
    - After adding Hydrogens, rename them "X" like in the previous step
- Check that each bond has the correct order. In the case of the PNS arm, make sure that the P has the correct bond orders to the oxygens
    - A double bond to one of the oxygens and single bonds to the rest
- Save as PNS.Mol2

[Result](Example_PNS.mol2)

[Result](Example2_PNS.mol2)

Use the following perl script (from [Justin Lemkul](http://www.mdtutorials.com/gmx/complex/Files/sort_mol2_bonds.pl)) to relist the bonds in ascending order

In [ ]:
perl sort_mol2_bonds.pl PNS.mol2 PNS_fix.mol2

[Result](Example_PNS_fix.mol2)

Upload PNS_fix.mol2 to [CGenff](https://cgenff.umaryland.edu/initguess/). Do not select any of the options before uploading. CGenFF will automatically rename all atoms with non-unique names (i.e all of the atoms named X)

<img src="Example_cgenff.png" alt="cgenff" width="400"/>

save the output as PNS.str (ParaChem CGenff toppar stream file)

[Result](Example_PNS.str)

### Convert .str file
The PNS.str file is currently in CHARMM format. We need to convert it to gromacs format before it will be useful to us. This can be accomplished using the python script [cgenff_charmm2gmx.py](http://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/cgenff_charmm2gmx.py) available at the [MacKerell Lab Site](http://mackerell.umaryland.edu/charmm_ff.shtml)
- If you created PNS_fix.mol2 on a remote computer, transfer it to the WSx workstation now
- We will be using the older .ff version here, this will not work with the new version

In [ ]:
!python2.7 cgenff_charmm2gmx.py PNS PNS_fix.mol2 PNS.str charmm36-jul2017.ff/

#### Possible Errors:
- Different versions of CGenFF----Solution: use an older forcefield
- "error in atomgroup.write_pdb(): atom name >	characters"
    - one or more of the atom names in the .str file has more than 4 characters

# Add Parameters to Charmm36
_**Note: Make all edits to current force field files, NOT old version**_

We now have all of the necessary parameters to build the PNS residue in charmm36. Our strategy will be to modify the existing SER residue in merged.rtp. We will keep the original parameters for the SER atoms and add in the arm atom parameters starting with the OG atom. This will create an issue with the parameters for the CB-OG bond as CGenff gives different atom types to the SER atoms. We will need to edit these parameters so there is a parameter for every bond listed in merged.rtp

Example: In PNS.str the CB atom type is CG321 and the OG atom type is OG303. As long as we only use the CGenff parameters, this is ok because there is a bond parameter in ffbonded.itp listed as CG321-OG303. However, we will be using the original SER atom parameters. This means that the CB atom type is now CT2. We will need to manually edit ffbonded.itp to include a parameter for CT2-OG303 (by copying and renaming the CG321-OG303 parameter). 

### Merged.rtp

Copy the SER residue (in merged.rtp) to the bottom of merged.rtp

```
...
[ SER ] 
  [ atoms ]
	    N   NH1   -0.470  0
	   HN     H    0.310  1
	   CA   CT1    0.070  2
	   HA   HB1    0.090  3
	   CB   CT2    0.050  4
	  HB1   HA2    0.090  5
	  HB2   HA2    0.090  6
	   OG   OH1   -0.660  7
	  HG1     H    0.430  8
	    C     C    0.510  9
	    O     O   -0.510 10
  [ bonds ]
	   CB    CA
	   OG    CB
	    N    HN
	    N    CA
	    C    CA
	    C    +N
	   CA    HA
	   CB   HB1
	   CB   HB2
	   OG   HG1
	    O     C
  [ impropers ]
	    N    -C    CA    HN
	    C    CA    +N     O
  [ cmap ]
	   -C     N    CA     C    +N
...
```

[Result](Example_merged.rtp)

Rename to [ PNS ]

- Replace OG atom with OG atom from PNS.str
- Delete HG1 atom (Hydrogen on OG of SER) and HG1-OG bond
- copy remaining non-backbone PNS atoms from PNS.str to merged.rtp

[Result](Example2_merged.rtp)

- Do the same for BONDS and Impropers

[Result](Example3_merged.rtp)

- format the new lines to match the old lines

[Result](Example4_merged.rtp)

### ffbonded.itp

ffbonded.itp contains all of the bond parameters that gromacs will use. CGenFF created many new bond parameters for the PNS arm and they have to be added to the file. 

We will also have to edit some manually. Because we connected the CGenff parameters from OG down to the original SER parameters, We have to rename some of the bond parameters.

First, copy all of the bond parameters in pns.prm to their respective sections in ffbonded.itp (I added the new parameters to the bottom of each section)

[Result](Example_ffbonded.itp)

Second, Identify all bond parameters that involve the CB-OG bond (this is where we are combining original SER parameters with CGenff parameters). Then translate the Atom names to the Atom types that CGenff has, then to the types that will be present in ffbonded.itp    

[Result](Example2_ffbonded.itp)

If there are multiple, intentionally rename one of the atom types with a random atomtype so there will be an error. gromacs will throw an error which you can use to tie the parameter to a bond

### Add PNS to residuetypes.dat

[Result](Example_residuetypes.dat)

# Gromacs 

We need .mdp files to follow the Gromacs steps. The .mdp files from Justin Lemkuls lysozyme tutorial are included in this directory

## Execute pdb2gmx
create a topology file, position restraint file, and a post-processed structure file

First double check that all PNS atom names in the pdb file match the atom names in the PNS entry in merged.rtp

In [ ]:
!echo 2| gmx pdb2gmx -f GmACP_PNS36.pdb  -o GmACPholo_processed.gro -water tip3p

Check the resulting GmACPholo_processed.gro in Chimera to make sure everything is connected

**Check the total charge of the protein, If it is not an integer value, the PNS entry in merged.rtp needs to be edited (if you are following this tutorial exactly, you WILL have to edit merged.rtp at this point)**
- To get to an integer charge, the charge of PNS needs to be adjusted by -0.124. I made this change to the beta carbon (CB). If you are applying the methods in this tutorial to a different novel residue, how to adjust the charge is something that will require carfeul thought
- After the charge is changed, re-run the pdb_2_gmx step

[Result](Example5_merged.rtp)

## Build a box around the protein

In [ ]:
!gmx editconf -f GmACPholo_processed.gro -o GmACPholo_newbox.gro -c -d 1.0 -bt cubic

## Add Solvent

In [ ]:
!gmx solvate -cp GmACPholo_newbox.gro -cs spc216.gro -o GmACPholo_solv.gro -p topol.top

## Add ions

In [ ]:
! cp /homes/cgavinhann/ACP/190225/ions.mdp ./

In [ ]:
!gmx grompp -f ions.mdp -c GmACPholo_solv.gro -p topol.top -o ions.tpr

_**Note:**_ It is common run into errors here. This is the first step that requires the bond parameters that we spent so much time editing
- Sometimes you will get warnings that parameters are being overridden
    - This is the result of duplicate parameters, duplicates can be commented out using a ";"
    - You should have this problem if you are following along exactly, I fixed it in the file linked in the next line
    - [Result](Example3_ffbonded.itp)
- "Note: non-zero total charge" (and not integer)
    - If the charge is something like "-4.999999) this is not an issue, There is a section about this in the gromacs documentation
- Re-run all steps starting from pdb_2_gmx once all issues are fixed

In [ ]:
!echo 13|gmx genion -s ions.tpr -o GmACPholo_solv_ions.gro -p topol.top -pname NA -nname CL -np 5

## Energy Minimization

In [ ]:
!gmx grompp -f minim.mdp -c GmACPholo_solv_ions.gro -p topol.top -o em.tpr

In [ ]:
!gmx mdrun -deffnm em

Check the output of energy minimization:

In [ ]:
! echo 1| gmx trjconv -f em.trr -s em.tpr -o em.pdb

In [ ]:
import py3Dmol; cut=1; pdbdata=open('em.pdb', 'r').read(); view=py3Dmol.view(); view.addModels(pdbdata,'pdb'); view.setStyle({'cartoon':{'color':'orange'}});  view.setStyle({'resi':'36'},{'stick':{'color':'orange'}});view.zoomTo(); view

- It is normal to see gaps between cartoon view (ribbon) and stick view when using py3Dmol

## Equilibration

In [ ]:
!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
!nohup gmx mdrun -deffnm nvt

In [ ]:
!gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
!nohup gmx mdrun -deffnm npt

## Production Run

In [ ]:
!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr

In [ ]:
!nohup gmx mdrun -deffnm md_0_1

In [ ]:
!echo 1 |gmx trjconv -f md_0_1.xtc -s md_0_1.tpr -o md_0_1.pdb -b 1000
#    - gives the final structure of the trajectory file

# Basic Analysis

In [ ]:
!(echo 1; echo 0)| gmx trjconv -f md_0_1.xtc -s md_0_1.tpr -o md_0_1_step1.xtc -pbc mol -center

In [ ]:
! (echo 1; echo 0)| gmx trjconv -f md_0_1.xtc -s md_0_1.tpr -o md_0_1_step1.gro -pbc mol -center -b 0 -e 0

In [ ]:
! (echo 1; echo 0)| gmx trjconv -s md_0_1.tpr -f md_0_1_step1.xtc -o md_0_1_step1.pdb -pbc mol -center -b 1000 -e 1000

In [ ]:
import py3Dmol
cut=1
pdbdata=open('md_0_1_step1.pdb', 'r').read()
view=py3Dmol.view()
view.addModel(pdbdata,'pdb')
chA = {'resi':'1-35'}
chB = {'resi':'36'}
chC = {'resi':'37-77'}
D = {'resi' : '78-171'}
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle(chA,{'cartoon':{'color':'spectrum'}})
view.setStyle(chC,{'cartoon':{'color':'spectrum'}})
view.setStyle(D,{'cartoon':{'color':'spectrum'}})
view.setStyle(chB, {'stick':{}})
#view.addSurface(py3Dmol.SAS,{'opacity':0.5,'colorscheme':{'prop':'b','gradient':'rwb','min':-cut,'max':cut}})
view

## MDAnalysis

In [ ]:
import MDAnalysis
import numpy as np
%matplotlib inline

### Build Universe

In [ ]:
u = MDAnalysis.Universe("md_0_1_step1.pdb", "md_0_1_step1.xtc")
ref = MDAnalysis.Universe("md_0_1_step1.pdb", "md_0_1_step1.xtc")

In [ ]:
print(u)

### RMS

In [ ]:
import MDAnalysis.analysis.rms

R = MDAnalysis.analysis.rms.RMSD(u, ref,
           select="protein or resname PNS",
           filename="rmsd.dat")
#if no ref is specified, current frame of atomgroup is used
R.run()

import matplotlib.pyplot as plt
rmsd = R.rmsd.T   # transpose makes it easier for plotting [1:] to remove first data point
time = rmsd[1]
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax.plot(time, rmsd[2], 'r--',  label="all")
ax.legend(loc="best")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"RMSD ($\AA$)")
plt.suptitle('RMS')
fig.savefig("rmsd.pdf")

In [ ]:
import matplotlib.pyplot as plt
plt.hist(rmsd[2], "auto", (rmsd[2].min(), rmsd[2].max()))  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

### Gyration

In [ ]:
Rgyr = []
protein = u.select_atoms("protein or resname PNS")
for ts in u.trajectory:
    Rgyr.append((u.trajectory.time, protein.radius_of_gyration()))
Rgyr = np.array(Rgyr)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt2
ax = plt2.subplot(111)
ax.plot(Rgyr[:,0], Rgyr[:,1], 'r--', lw=2, label=r"$R_G$")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"radius of gyration $R_G$ ($\AA$)")
ax.figure.savefig("Rgyr.pdf")
plt.suptitle('Radius of Gyration (Protein)')
plt.draw()

# Sources

[CGenff Website](https://cgenff.umaryland.edu)

[Gromacs Tutorial: Protein-Ligand Complex by Justin Lemkul, PhD](http://www.mdtutorials.com/gmx/complex/index.html)

[Adding a Residue to a Force Field](http://www.gromacs.org/Documentation/How-tos/Adding_a_Residue_to_a_Force_Field) (Gromacs help pages)